In [ ]:
from pyspark.sql import SparkSession, DataFrame
from pyspark.sql.types import *
from pyspark.sql.functions import col, concat_ws, row_number
from pyspark.sql.window import Window
from itertools import chain
from typing import Dict
import re 
from pyspark.sql.functions import col, lit, when, coalesce, split, concat, udf, \
                                    regexp_replace, lower, monotonically_increasing_id, regexp_extract, create_map
from pyspark.sql.functions import sum, col, month, year, format_number 


In [ ]:
# Add PostgreSQL JDBC driver to the Spark session
spark = SparkSession.builder \
    .appName("Spark_postgres_metastore") \
    .config("spark.jars", "/opt/spark3/jars/postgresql-42.2.19.jar") \
    .getOrCreate()


In [ ]:
# JDBC connection properties
jdbc_url = "jdbc:postgresql://metabase_db:5432/metabase"
connection_properties = {
    "user": "metabase",
    "password": "metabasepassword",
    "driver": "org.postgresql.Driver"
}

In [ ]:
# Define schema for date_dim
date_dim_schema = StructType([
    StructField("datee", DateType(), True),
    StructField("date_key", LongType(), True),
    StructField("day", IntegerType(), True),
    StructField("day_of_week", IntegerType(), True),
    StructField("day_of_year", IntegerType(), True),
    StructField("week_of_year", IntegerType(), True),
    StructField("month", IntegerType(), True),
    StructField("quarter", IntegerType(), True),
    StructField("year", IntegerType(), True),
    StructField("is_weekend", IntegerType(), True),
    StructField("month_name", StringType(), True),
    StructField("day_name", StringType(), True),
    StructField("year_month", StringType(), True)
])

# Load date_dim data
date_dim_df = spark.read.csv("/Graduation_Project/transformed_data/schema/date_dim.csv/", 
                             schema=date_dim_schema, header=True)

In [ ]:
# Define schema for branches_dim
branches_dim_schema = StructType([
    StructField("branch_id", DoubleType(), True),
    StructField("branch_location", StringType(), True),
    StructField("branch_establish_date", DateType(), True),
    StructField("branch_class", StringType(), True),
    StructField("branch_key", IntegerType(), True)
])

# Load branches_dim data
branches_dim_df = spark.read.csv("/Graduation_Project/transformed_data/schema/branches_dim.csv/", 
                                 schema=branches_dim_schema, header=True)

In [ ]:
# Define schema for sales_agent_dim
sales_agent_dim_schema = StructType([
    StructField("sales_agent_id", DoubleType(), True),
    StructField("sales_agent_hire_date", DateType(), True),
    StructField("sales_agent_name", StringType(), True),
    StructField("sales_agent_key", IntegerType(), True)
])

# Load sales_agent_dim data
sales_agent_dim_df = spark.read.csv("/Graduation_Project/transformed_data/schema/sales_agent_dim.csv/", 
                                    schema=sales_agent_dim_schema, header=True)

In [ ]:
# Define schema for customer_dim
customer_dim_schema = StructType([
    StructField("customer_id", StringType(), True),
    StructField("customer_name", StringType(), True),
    StructField("cleaned_email", StringType(), True),
    StructField("customer_key", IntegerType(), True)
])

# Load customer_dim data
customer_dim_df = spark.read.csv("/Graduation_Project/transformed_data/schema/customer_dim.csv/", 
                                 schema=customer_dim_schema, header=True)

In [ ]:
# Define schema for product_dim
product_dim_schema = StructType([
    StructField("product_id", StringType(), True),
    StructField("product_name", StringType(), True),
    StructField("product_category", StringType(), True),
    StructField("unit_price", DoubleType(), True),
    StructField("product_key", IntegerType(), True)
])

# Load product_dim data
product_dim_df = spark.read.csv("/Graduation_Project/transformed_data/schema/products_dim.csv/", 
                                schema=product_dim_schema, header=True)

In [ ]:
# Define schema for offline_transactions_fact
offline_transactions_fact_schema = StructType([
    StructField("transaction_id", StringType(), True),
    StructField("units", IntegerType(), True),
    StructField("discount", DoubleType(), True),
    StructField("payment_method", StringType(), True),
    StructField("total_price", DoubleType(), True),
    StructField("date_key", LongType(), True),
    StructField("product_key", IntegerType(), True),
    StructField("customer_name", StringType(), True),
    StructField("cleaned_email", StringType(), True),
    StructField("customer_key", IntegerType(), True),
    StructField("branch_key", IntegerType(), True),
    StructField("sales_agent_key", IntegerType(), True)
])

# Load offline_transactions_fact data
offline_transactions_fact_df = spark.read.csv("/Graduation_Project/transformed_data/schema/offline_transactions_fact.csv/", 
                                              schema=offline_transactions_fact_schema, header=True)

In [ ]:
# Define schema for online_transactions_fact
online_transactions_fact_schema = StructType([
    StructField("transaction_id", StringType(), True),
    StructField("units", IntegerType(), True),
    StructField("discount", DoubleType(), True),
    StructField("payment_method", StringType(), True),
    StructField("group_from", IntegerType(), True),
    StructField("total_price", DoubleType(), True),
    StructField("date_key", LongType(), True),
    StructField("shipping_zip_code", StringType(), True),
    StructField("shipping_state", StringType(), True),
    StructField("shipping_city", StringType(), True),
    StructField("shipping_street_name", StringType(), True),
    StructField("product_key", IntegerType(), True),
    StructField("customer_key", IntegerType(), True)
])

# Load online_transactions_fact data
online_transactions_fact_df = spark.read.csv("/Graduation_Project/transformed_data/schema/online_transactions_fact.csv/", 
                                             schema=online_transactions_fact_schema, header=True)

In [178]:
# Join online and offline transactions with date_dim to get the month and year
online_sales_with_date = online_transactions_fact_df.join(date_dim_df, online_transactions_fact_df['date_key'] == date_dim_df['date_key'])
offline_sales_with_date = offline_transactions_fact_df.join(date_dim_df, offline_transactions_fact_df['date_key'] == date_dim_df['date_key'])

# Calculate total sales by month for online transactions 
online_monthly_sales = online_sales_with_date.groupBy(year('datee').alias('year'), month('datee').alias('month')).agg(sum('total_price').alias('total_sales'))

# Calculate total sales by month for offline transactions
offline_monthly_sales = offline_sales_with_date.groupBy(year('datee').alias('year'), month('datee').alias('month')).agg(sum('total_price').alias('total_sales'))

# Combine both online and offline sales
total_monthly_sales_2 = online_monthly_sales.union(offline_monthly_sales).groupBy('year', 'month').agg(sum('total_sales').alias('total_sales')).orderBy('year', 'month')


# Show the result
total_monthly_sales_2.show()


+----+-----+--------------------+
|year|month|         total_sales|
+----+-----+--------------------+
|2022|    1| 2.240577144306126E7|
|2022|    2| 2.250910127161768E7|
|2022|    3| 2.201956558332828E7|
|2022|    4| 2.188690018374264E7|
|2022|    5|2.2693279069808323E7|
|2022|    6|2.2266633558739174E7|
|2022|    7|2.3118430164284844E7|
|2022|    8|2.2497213684118897E7|
|2022|    9|2.2234833784322567E7|
|2022|   10| 2.267815755658198E7|
|2022|   11|2.2267563414613158E7|
|2022|   12|2.2834241794397544E7|
|2023|    1|2.2466372007944487E7|
|2023|    2| 2.247959145307955E7|
|2023|    3| 2.270246326197837E7|
|2023|    4|2.2601553327856522E7|
|2023|    5|2.2439523122745883E7|
|2023|    6|2.2933544736905064E7|
|2023|    7| 2.244627951684256E7|
|2023|    8|2.2465396811820537E7|
+----+-----+--------------------+
only showing top 20 rows



In [ ]:
# Format the total_sales column
total_monthly_sales = total_monthly_sales_2.withColumn('total_sales', format_number('total_sales', 2))
total_monthly_sales.show()



In [162]:
# Write DataFrame to PostgreSQL
total_monthly_sales.write.jdbc(url=jdbc_url, table="total_monthly_sales", mode="overwrite", properties=connection_properties)

In [179]:
# Join transactions with product_dim
online_sales_with_product = online_transactions_fact_df.join(product_dim_df, online_transactions_fact_df['product_key'] == product_dim_df['product_key'])
offline_sales_with_product = offline_transactions_fact_df.join(product_dim_df, offline_transactions_fact_df['product_key'] == product_dim_df['product_key'])

# Calculate total sales by product for online transactions
online_product_sales = online_sales_with_product.groupBy('product_name').agg(sum('total_price').alias('total_sales'))

# Calculate total sales by product for offline transactions
offline_product_sales = offline_sales_with_product.groupBy('product_name').agg(sum('total_price').alias('total_sales'))

# Combine both online and offline sales
total_product_sales = online_product_sales.union(offline_product_sales).groupBy('product_name').agg(sum('total_sales').alias('total_sales')).orderBy('total_sales', ascending=False)

# Format the total_sales column
total_product_sales = total_product_sales.withColumn('total_sales', format_number('total_sales', 2))

# Show the result
total_product_sales.show()


+-----------------+-------------+
|     product_name|  total_sales|
+-----------------+-------------+
|           Laptop|96,558,184.53|
|               TV|84,177,639.80|
|       Smartphone|67,389,192.37|
|  Washing Machine|47,582,798.38|
|           Camera|37,987,410.34|
|           Tablet|28,426,137.47|
|          Monitor|28,305,661.48|
|            Boots|26,670,636.36|
|   Vacuum Cleaner|18,980,950.93|
|          Printer|14,240,140.61|
|        Microwave|12,448,407.87|
|       Headphones| 9,564,033.51|
|         Sneakers| 7,623,151.00|
|     Coffee Maker| 7,494,055.14|
|          Sandals| 6,664,337.03|
|            Dress| 5,715,028.34|
|            Heels| 5,710,850.04|
|          Blender| 4,771,288.06|
|            Jeans| 4,715,621.69|
|Hair Straightener| 3,847,495.89|
+-----------------+-------------+
only showing top 20 rows



In [163]:
# Write DataFrame to PostgreSQL
total_product_sales.write.jdbc(url=jdbc_url, table="total_product_sales", mode="overwrite", properties=connection_properties)

In [180]:
# Calculate total sales by payment method for online transactions
online_payment_method_sales = online_transactions_fact_df.groupBy('payment_method').agg(sum('total_price').alias('total_sales'))

# Calculate total sales by payment method for offline transactions
offline_payment_method_sales = offline_transactions_fact_df.groupBy('payment_method').agg(sum('total_price').alias('total_sales'))

# Combine both online and offline sales
total_payment_method_sales = online_payment_method_sales.union(offline_payment_method_sales).groupBy('payment_method').agg(sum('total_sales').alias('total_sales')).orderBy('total_sales', ascending=False)

# Format the total_sales column
total_payment_method_sales = total_payment_method_sales.withColumn('total_sales', format_number('total_sales', 2))


total_payment_method_sales.show()


+--------------+--------------+
|payment_method|   total_sales|
+--------------+--------------+
|   Credit Card|262,511,822.18|
|          Cash|245,202,090.97|
|        PayPal| 16,836,704.48|
|        Stripe| 16,698,372.44|
+--------------+--------------+



In [164]:
# Write DataFrame to PostgreSQL
total_payment_method_sales.write.jdbc(url=jdbc_url, table="total_payment_method_sales", mode="overwrite", properties=connection_properties)

In [ ]:
# Join transactions with customer_dim
offline_transactions_fact_df_e=offline_transactions_fact_df.drop("customer_name")
online_sales_with_customer = online_transactions_fact_df.join(customer_dim_df, online_transactions_fact_df['customer_key'] == customer_dim_df['customer_key'])
offline_sales_with_customer = offline_transactions_fact_df_e.join(customer_dim_df, offline_transactions_fact_df['customer_key'] == customer_dim_df['customer_key'])

# Calculate total sales by customer for online transactions
online_customer_sales = online_sales_with_customer.groupBy('customer_id').agg(sum('total_price').alias('total_sales'))

# Calculate total sales by customer for offline transactions
offline_customer_sales = offline_sales_with_customer.groupBy('customer_id').agg(sum('total_price').alias('total_sales'))

# Combine both online and offline sales
total_customer_sales = online_customer_sales.union(offline_customer_sales).groupBy('customer_id').agg(sum('total_sales').alias('total_sales')).orderBy('total_sales', ascending=False)

# Format the total_sales column
total_customer_sales = total_customer_sales.withColumn('total_sales', format_number('total_sales', 2))

total_customer_sales.show(101)

In [165]:
# Write DataFrame to PostgreSQL
total_customer_sales.write.jdbc(url=jdbc_url, table="total_customer_sales", mode="overwrite", properties=connection_properties)

In [ ]:
# Join offline transactions with branches_dim
offline_sales_with_branch = offline_transactions_fact_df.join(branches_dim_df, offline_transactions_fact_df['branch_key'] == branches_dim_df['branch_key'])

# Calculate total sales by branch
branch_sales = offline_sales_with_branch.groupBy('branch_id').agg(sum('total_price').alias('total_sales')).orderBy('total_sales', ascending=False)

# Format the total_sales column
branch_sales = branch_sales.withColumn('total_sales', format_number('total_sales', 2))

branch_sales.show()


In [166]:
# Write DataFrame to PostgreSQL
branch_sales.write.jdbc(url=jdbc_url, table="branch_sales", mode="overwrite", properties=connection_properties)

In [ ]:
# Join offline transactions with sales_agent_dim
offline_sales_with_agent = offline_transactions_fact_df.join(sales_agent_dim_df, offline_transactions_fact_df['sales_agent_key'] == sales_agent_dim_df['sales_agent_key'])

# Calculate total sales by sales agent
agent_sales = offline_sales_with_agent.groupBy('sales_agent_name').agg(sum('total_price').alias('total_sales')).orderBy('total_sales', ascending=False)

# Format the total_sales column
agent_sales = agent_sales.withColumn('total_sales', format_number('total_sales', 2))

agent_sales.show()


In [167]:
# Write DataFrame to PostgreSQL
agent_sales.write.jdbc(url=jdbc_url, table="agent_sales", mode="overwrite", properties=connection_properties)

In [ ]:
# Calculate units sold by product for online transactions
online_units_by_product = online_transactions_fact_df.groupBy('product_key').agg(sum('units').alias('total_units'))

# Calculate units sold by product for offline transactions
offline_units_by_product = offline_transactions_fact_df.groupBy('product_key').agg(sum('units').alias('total_units'))

# Combine both online and offline units sold
total_units_by_product = online_units_by_product.union(offline_units_by_product).groupBy('product_key').agg(sum('total_units').alias('total_units')).orderBy('total_units', ascending=False).limit(10)

# Join with product_dim to get product names
top_products = total_units_by_product.join(product_dim_df, total_units_by_product['product_key'] == product_dim_df['product_key']).select('product_name', 'total_units').orderBy('total_units', ascending=False)

# Format the total_sales column
top_products = top_products.withColumn('total_units', format_number('total_units', 2))

top_products.show()


In [168]:
# Write DataFrame to PostgreSQL
top_products.write.jdbc(url=jdbc_url, table="top_products", mode="overwrite", properties=connection_properties)

In [ ]:
# Calculate average discount by product for online transactions
online_discount_by_product = online_transactions_fact_df.groupBy('product_key').agg(avg('discount').alias('average_discount'))

# Calculate average discount by product for offline transactions
offline_discount_by_product = offline_transactions_fact_df.groupBy('product_key').agg(avg('discount').alias('average_discount'))

# Combine both online and offline average discounts
total_discount_by_product = online_discount_by_product.union(offline_discount_by_product).groupBy('product_key').agg(avg('average_discount').alias('average_discount')).orderBy('average_discount', ascending=False)

# Join with product_dim to get product names
discount_by_product = total_discount_by_product.join(product_dim_df, total_discount_by_product['product_key'] == product_dim_df['product_key']).select('product_name', 'average_discount').orderBy('average_discount', ascending=False)

# Format the total_sales column
discount_by_product = discount_by_product.withColumn('average_discount', format_number('average_discount', 5))


discount_by_product.show()


In [169]:
# Write DataFrame to PostgreSQL
discount_by_product.write.jdbc(url=jdbc_url, table="discount_by_product", mode="overwrite", properties=connection_properties)

In [ ]:
# Calculate sales trend over time for online transactions
online_sales_trend = online_transactions_fact_df.join(date_dim_df, online_transactions_fact_df['date_key'] == date_dim_df['date_key']).groupBy('datee').agg(sum('total_price').alias('total_sales')).orderBy('datee')

# Calculate sales trend over time for offline transactions
offline_sales_trend = offline_transactions_fact_df.join(date_dim_df, offline_transactions_fact_df['date_key'] == date_dim_df['date_key']).groupBy('datee').agg(sum('total_price').alias('total_sales')).orderBy('datee')

# Combine both online and offline sales trends
total_sales_trend = online_sales_trend.union(offline_sales_trend).groupBy('datee').agg(sum('total_sales').alias('total_sales')).orderBy('datee')

# Format the total_sales column
total_sales_trend = total_sales_trend.withColumn('total_sales', format_number('total_sales', 2))



total_sales_trend.show()


In [170]:
# Write DataFrame to PostgreSQL
total_sales_trend.write.jdbc(url=jdbc_url, table="total_sales_trend", mode="overwrite", properties=connection_properties)

In [ ]:
# Join transactions with product_dim
online_sales_with_category = online_transactions_fact_df.join(product_dim_df, online_transactions_fact_df['product_key'] == product_dim_df['product_key'])
offline_sales_with_category = offline_transactions_fact_df.join(product_dim_df, offline_transactions_fact_df['product_key'] == product_dim_df['product_key'])

# Calculate total sales by product category for online transactions
online_category_sales = online_sales_with_category.groupBy('product_category').agg(sum('total_price').alias('total_sales'))

# Calculate total sales by product category for offline transactions
offline_category_sales = offline_sales_with_category.groupBy('product_category').agg(sum('total_price').alias('total_sales'))

# Combine both online and offline sales
total_category_sales = online_category_sales.union(offline_category_sales).groupBy('product_category').agg(sum('total_sales').alias('total_sales')).orderBy('total_sales', ascending=False)

# Format the total_sales column
total_category_sales = total_category_sales.withColumn('total_sales', format_number('total_sales', 2))


total_category_sales.show()


In [171]:
# Write DataFrame to PostgreSQL
total_category_sales.write.jdbc(url=jdbc_url, table="total_category_sales", mode="overwrite", properties=connection_properties)

In [ ]:
# Calculate average sales per customer for online transactions
online_avg_sales_per_customer = online_transactions_fact_df.groupBy('customer_key').agg(avg('total_price').alias('average_sales'))

# Calculate average sales per customer for offline transactions
offline_avg_sales_per_customer = offline_transactions_fact_df.groupBy('customer_key').agg(avg('total_price').alias('average_sales'))

# Combine both online and offline average sales per customer
total_avg_sales_per_customer = online_avg_sales_per_customer.union(offline_avg_sales_per_customer).groupBy('customer_key').agg(avg('average_sales').alias('average_sales')).orderBy('average_sales', ascending=False)

# Join with customer_dim to get customer names
avg_sales_per_customer = total_avg_sales_per_customer.join(customer_dim_df, total_avg_sales_per_customer['customer_key'] == customer_dim_df['customer_key']).select('customer_name', 'average_sales').orderBy('average_sales', ascending=False)

# Format the total_sales column
avg_sales_per_customer = avg_sales_per_customer.withColumn('average_sales', format_number('average_sales', 2))


avg_sales_per_customer.show()


In [172]:
# Write DataFrame to PostgreSQL
avg_sales_per_customer.write.jdbc(url=jdbc_url, table="avg_sales_per_customer", mode="overwrite", properties=connection_properties)

In [ ]:
# Calculate frequency of transactions by customer for online transactions
online_freq_customers = online_transactions_fact_df.groupBy('customer_key').count()

# Calculate frequency of transactions by customer for offline transactions
offline_freq_customers = offline_transactions_fact_df.groupBy('customer_key').count()

# Combine both online and offline frequency of transactions
total_freq_customers = online_freq_customers.union(offline_freq_customers).groupBy('customer_key').agg(sum('count').alias('total_transactions'))

# Join with customer_dim to get customer names
freq_customers = total_freq_customers.join(customer_dim_df, 'customer_key').select('customer_name', 'total_transactions').orderBy('total_transactions', ascending=False)

# Show the result
freq_customers.show()


In [173]:
# Write DataFrame to PostgreSQL
freq_customers.write.jdbc(url=jdbc_url, table="freq_customers", mode="overwrite", properties=connection_properties)

In [ ]:
# Calculate total sales by state for online transactions
online_sales_by_state = online_transactions_fact_df.groupBy('shipping_state').agg(sum('total_price').alias('total_sales'))

# Calculate total sales by state for offline transactions
offline_sales_by_state = offline_transactions_fact_df.join(branches_dim_df, offline_transactions_fact_df['branch_key'] == branches_dim_df['branch_key']).groupBy('branch_location').agg(sum('total_price').alias('total_sales'))

# Combine both online and offline sales by state
total_sales_by_state = online_sales_by_state.union(offline_sales_by_state).groupBy('shipping_state').agg(sum('total_sales').alias('total_sales')).orderBy('total_sales', ascending=False).limit(10)

# Format the total_sales column
total_sales_by_state = total_sales_by_state.withColumn('total_sales', format_number('total_sales', 2))

total_sales_by_state.show()


In [174]:
# Write DataFrame to PostgreSQL
total_sales_by_state.write.jdbc(url=jdbc_url, table="total_sales_by_state", mode="overwrite", properties=connection_properties)

In [ ]:
# Calculate total sales and total units by customer for online transactions
online_clv = online_transactions_fact_df.groupBy('customer_key').agg(sum('total_price').alias('total_sales'), sum('units').alias('total_units'))

# Calculate total sales and total units by customer for offline transactions
offline_clv = offline_transactions_fact_df.groupBy('customer_key').agg(sum('total_price').alias('total_sales'), sum('units').alias('total_units'))

# Combine both online and offline CLV
total_clv = online_clv.union(offline_clv).groupBy('customer_key').agg(sum('total_sales').alias('total_sales'), sum('total_units').alias('total_units')).orderBy('total_sales', ascending=False)

# Join with customer_dim to get customer names
clv = total_clv.join(customer_dim_df, total_clv['customer_key'] == customer_dim_df['customer_key']).select('customer_name', 'total_sales', 'total_units').orderBy('total_sales', ascending=False)

# Format the total_sales column
clv = clv.withColumn('total_sales', format_number('total_sales', 2))


clv.show()


In [175]:
# Write DataFrame to PostgreSQL
clv.write.jdbc(url=jdbc_url, table="clv", mode="overwrite", properties=connection_properties)

In [ ]:
from pyspark.sql.functions import lag
from pyspark.sql.window import Window

# Calculate total sales by month
monthly_sales = total_monthly_sales_2

# Add a column for the previous month's sales
window_spec = Window.orderBy('year', 'month')
monthly_sales = monthly_sales.withColumn('prev_month_sales', lag('total_sales').over(window_spec))

# Calculate the growth rate
monthly_sales = monthly_sales.withColumn('growth_rate', (col('total_sales') - col('prev_month_sales')) / col('prev_month_sales') * 100)

# Format the total_sales column
monthly_sales = monthly_sales.withColumn('growth_rate', format_number('growth_rate', 2))

monthly_sales.show()


In [177]:
# Write DataFrame to PostgreSQL
monthly_sales.write.jdbc(url=jdbc_url, table="monthly_sales", mode="overwrite", properties=connection_properties)